In [14]:
! pip install 'git+https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git'

  Cloning https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-4fukckzw
  Running command git clone -q https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-4fukckzw
  Resolved https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to commit ad04061f3ac4769343da72cb72115ecad787aa2c


In [15]:
import JUSTFAIR_Tools as jt

In [18]:
jt.toolbox.hello_world()

AttributeError: module 'JUSTFAIR_Tools' has no attribute 'toolbox'

In [16]:
dir(jt)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']